# Library

In [12]:
import os
import json
import time

from dotenv import load_dotenv
from typing import Optional, Dict, Any
from pydantic import BaseModel, EmailStr, ValidationError
from openai import OpenAI

load_dotenv()

True

In [4]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [5]:
client = OpenAI(api_key=OPENAI_API_KEY)
client

In [13]:
class Lead(BaseModel):
    name: str
    email: EmailStr
    phone: Optional[str] = None
    company: Optional[str] = None
    notes: Optional[str] = None

In [ ]:
CREATE_LEAD_SCHEMA = {
    "type": "function",
    "name": "create_lead",
    "description": "Any new lead that is created in the CRM",
    "parameters": {
        "type": "object",
        "properties": {
            "name": {
                "type": "string",
                "description": "The name of the lead"
            },
            "email": {
                "type": "string",
                "format": "email",
                "description": "The email address of the lead"  
            },
            "phone": {
                "type": ["string", "null"],
                "description": "The phone number of the lead"
            },
            "company": {
                "type": ["string", "null"],
                "description": "The company of the lead"
            },
            "notes": {
                "type": ["string", "null"],
                "description": "Any additional notes about the lead"
            }
        },
        "required": ["name", "email"],
        "additionalProperties": False
    },
    "strict": True
}

In [31]:
PROMPT_CREATE_LEAD = """
You are an assistant of a CRM system. Your task is to help users create new leads by providing the necessary information and guidance.
Return a JSON following the lead_schema, do not include any additional information or context.
If the user provides all the required information, create the lead. If any information is missing, prompt the user to provide the missing information.
"""

In [ ]:
def call_llm_with_retry(
    prompt: str,
    schema: Dict[str, Any],
    validator: Optional[Any] = None,
    max_retries: int = 3,
    retry_delay: float = 1.0,
    **kwargs: Any
) -> BaseModel: 

    for attempt in range(max_retries):
        try:
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "user", "content": prompt},
                    {"role": "system", "content": "You are a helpful assistant."}
                ],
                functions=[schema],
                function_call={"name": schema["name"]},
                **kwargs
            )
            response_message = response.choices[0].message
            if response_message.function_call:
                function_args = json.loads(response_message.function_call.arguments)

                if validator:
                    obj = validator(**function_args)
                    return obj
                else:
                    return function_args
            else:
                raise ValueError("No function call found in the response.")
        except (ValidationError, ValueError) as e:
            print(f"[Retry {attempt + 1}] Validation error: {e}. Retrying...")
            if attempt < max_retries - 1:
                prompt += f"\nPlease provide the missing information. Please ensure all required fields are filled. {json.dumps(schema)}"
                time.sleep(retry_delay)
            else:
                raise
    raise RuntimeError("Max retries exceeded without a valid response.")

In [61]:
def mock_crm_api_create_lead(data: dict) -> dict:
    """Fake API để tạo Lead"""
    print(f"[MOCK CRM API] Đã tạo Lead: {data.name} - {data.email}")
    return {"status": "success", "lead_id": "LEAD12345"}

In [65]:
user_input = "Create new Lead with name 'Nguyễn Văn A', email 'a@example.com', phone '0987654321', company 'Skymap'"

lead_data = call_llm_with_retry(PROMPT_CREATE_LEAD + "\n" + user_input, CREATE_LEAD_SCHEMA, validator=Lead)

print(lead_data)

if lead_data:
    result = mock_crm_api_create_lead(lead_data)
    print(f"Lead created successfully: {result}")
else:
    print("Failed to create lead.")

name='Nguyễn Văn A' email='a@example.com' phone='0987654321' company='Skymap' notes=None
[MOCK CRM API] Đã tạo Lead: Nguyễn Văn A - a@example.com
Lead created successfully: {'status': 'success', 'lead_id': 'LEAD12345'}
